In [9]:
import pandas as pd
import numpy as np
utility_matrix = pd.read_csv("50%_utility_matrix.csv")
feature_matrix = pd.read_csv("feature_matrix.csv")
test_set = pd.read_csv("testing_set.csv")

In [ ]:
#create dictionary to map anime_id to its genre vector
feature_matrix_indexed = feature_matrix.set_index('anime_id')
anime_vector = feature_matrix_indexed['genre_vector'].to_dict()
for key in anime_vector.keys():
    anime_vector[key] = eval(anime_vector[key])
print(anime_vector)

In [12]:
source_table = {"train" : utility_matrix,"test" : test_set}
def getRatingVectorOfNthUser(n:int,source):
    #get the rating vector of n-th user and the min and max anime_id
    column_n = str(n)
    rating = source_table[source][[column_n]].values
    not_minus1_sum,count = 0,0
    for i in rating:
        if i[0] != -1 and i[0] != 0:
            not_minus1_sum += i[0]
            count += 1
    if count != 0:
        mean = round(not_minus1_sum/count,2)
        for i in range(len(rating)):
            if rating[i][0] == -1:
                rating[i][0] = mean
    else:
        for i in range(len(rating)):
            if rating[i][0] == -1:
                rating[i][0] = 5
        
    #normalize the rating value
    del not_minus1_sum,count,column_n
    
    
    return rating

print(getRatingVectorOfNthUser(1,"train"))

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [5]:
#u1 and u2 must be array
def similarity(u1,u2) -> float:
    
    
    return ((u1.T).dot(u2))/((np.linalg.norm(u1))*np.linalg.norm(u2))
u1 = getRatingVectorOfNthUser(1,"train")
u2 = getRatingVectorOfNthUser(10,"train")
print(similarity(u1,u2)[0][0])

0.11224168986796883


In [6]:
#calculate the similarity matrix of all users in our dataset
#this is not necessary for the prediction but  can be useful to cluster the users into distinct groups
similar_matrix = [[0 for i in range(1000)] for j in range(1000)]
print(np.array(similar_matrix))
for i in range(len(similar_matrix)):
    for j in range(len(similar_matrix)):
        if i == j:
            similar_matrix[i][j] = 1
        if similar_matrix[j][i] != 0:
            similar_matrix[i][j] = similar_matrix[j][i]
        else:
            similar_matrix[i][j] = similarity(i+1,j+1)[0][0]
print(np.array(similar_matrix))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[ 1.         -0.04225847  0.01613407 ... -0.03050808  0.01968847
   0.11480223]
 [-0.04225847  1.          0.13368172 ...  0.          0.
   0.        ]
 [ 0.01613407  0.13368172  1.         ...  0.05790612  0.2762701
   0.1151693 ]
 ...
 [-0.03050808  0.          0.05790612 ...  1.          0.09421744
   0.06584984]
 [ 0.01968847  0.          0.2762701  ...  0.09421744  1.
   0.13113166]
 [ 0.11480223  0.          0.1151693  ...  0.06584984  0.13113166
   1.        ]]


In [8]:
user = 1
max_similarity = 0
list_of_max = []
for i in range(2,36758):
    if similarity(user,i)[0][0] > max_similarity:
        max_similarity = similarity(user,i)[0][0]
        list_of_max.append((i,max_similarity))
print(list_of_max[:-4])

[(2, 0.046676002800933654), (3, 0.1480248916078675), (5, 0.16796235193930292), (7, 0.2676526129167347), (27, 0.2679275009052405), (41, 0.3719707226219281)]


In [7]:
def all_zero(arr):
    for i in arr:
        if i != 0:
            return False
    return True
print(all_zero(getRatingVectorOfNthUser(10,"train")))


False


In [ ]:
test_user = None
user_vector = getRatingVectorOfNthUser(test_user)
list_of_similarity = []
max_sim = 0
if all_zero(user_vector):
    #recommend the most popular item if user has no rated items yet.
    continue
else:
    for train_user in dataset:
        train_vector = getRatingVectorOfNthUser(train_user)
        if all_zero(train_vector):
            continue
        else:
            if similarity(user_vector, train_vector)[0][0] > max_sim:
                max_sim = similarity(user_vector, train_vector)[0][0]
                list_of_similarity.append(train_user,max_sim)

lấy ~1000 users mới để làm testing set : done
hàm getRatingVectorOfNthUser(user:int) nên đổi thành  hàm getRatingVectorOfUser(user:int, source = "test" or source = "train") để phân biệt. : done 